In [1]:
from ensmallen import HyperSketchingPy

2024-01-22 15:23:27.520877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import grape
from grape import Graph

INFO:pykeen.utils:Using opt_einsum


In [3]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
from cache_decorator import Cache

In [4]:
graph = Graph.from_csv(
    name="LOTUS_with_NCBITaxon",
    node_path="./data/lotus_with_ncbi_nodes.csv",
    node_list_separator="\t",
    node_list_header=True,
    nodes_column_number=0,
    node_list_node_types_column_number=1,
    edge_path="./data/lotus_with_ncbi_edges.csv",
    edge_list_separator="\t",
    edge_list_header=True,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_edge_types_column_number=2,
    #directed=True,
    directed=False,
)
graph = graph.remove_singleton_nodes()
graph = graph.remove_components(top_k_components=1)

In [5]:
#graph.get_diameter()

*graph diamter* = 44

44*44 + 2 * 44 is maximum number of features of the graph

In [6]:
44*44+2*44

2024

commbination : 
- "addition",
- "subtraction",
- "manhattan",
- "euclidean",
- "hadamard",
- "concatenate",

In [7]:
normalize = False
combination = "addition"
number_of_hops = 2

In [8]:
train, test = graph.get_edge_prediction_kfold(
    k_index=0, 
    k=5,
    edge_types=["biolink:in_taxon"],
    random_state=42,
)

In [22]:
filtered_train = train.filter_from_names(
    edge_type_names_to_keep=["biolink:in_taxon"],
)

subtrain, validation = train.(
    train_size= 0.8,
    edge_types=["biolink:in_taxon"],
    random_state=42,
)

In [13]:
negative = graph.sample_negative_graph(
    source_edge_types_names=["biolink:in_taxon"],
    destination_edge_types_names=["biolink:in_taxon"],
    number_of_negative_samples=graph.get_number_of_edges_from_edge_type_name("biolink:in_taxon")*2,
    sample_edge_types=False,
    only_from_same_component=False,
    use_scale_free_distribution=True,
    random_state=23391 * (10 + 1)
)
neg_train, neg_test = negative.random_holdout(train_size=0.8)

In [15]:
graph.get_number_of_edges_from_edge_type_name("biolink:in_taxon")

1066694

In [16]:
%%time 
sketching_features = HyperSketchingPy(
    hops=number_of_hops,
    normalize=normalize,
    graph=train
)
sketching_features.fit()

CPU times: user 2min 3s, sys: 49 s, total: 2min 52s
Wall time: 1min 10s


In [23]:
pos_sources = filtered_train.get_directed_source_node_ids()
pos_destinations = filtered_train.get_directed_destination_node_ids()

In [24]:
%%time
sk_positive_features = sketching_features.positive(
    sources=pos_sources,
    destinations=pos_destinations,
    feature_combination=combination
)

CPU times: user 5min 4s, sys: 1.91 s, total: 5min 6s
Wall time: 1min 23s


In [28]:
import numpy as np

In [30]:
np.isnan(sk_positive_features).any()

False

In [32]:
np.save("./sk_positive_features.npy",
        sk_positive_features)

In [25]:
pos_sources.shape

(1013358,)

In [33]:
%%time 
neg_sources = neg_train.get_directed_source_node_ids()
neg_destinations = neg_train.get_directed_destination_node_ids()
sk_negative_features = sketching_features.negative(
    sources=neg_sources,
    destinations=neg_destinations,
    feature_combination=combination
)

CPU times: user 8min 14s, sys: 3.24 s, total: 8min 17s
Wall time: 2min 28s


In [40]:
%%time
pos_test_srcs = test.get_directed_source_node_ids()
pos_test_dsts = test.get_directed_destination_node_ids()
sk_test_pos_features = sketching_features.unknown(
    sources=pos_test_srcs,
    destinations=pos_test_dsts,
    feature_combination=combination
)

CPU times: user 27.9 s, sys: 230 ms, total: 28.1 s
Wall time: 9.32 s


In [41]:
%%time
neg_test_srcs = neg_test.get_directed_source_node_ids()
neg_test_dsts = neg_test.get_directed_destination_node_ids()
sk_test_neg_features = sketching_features.unknown(
    sources=neg_test_srcs,
    destinations=neg_test_dsts,
    feature_combination=combination
)

CPU times: user 3min 29s, sys: 907 ms, total: 3min 30s
Wall time: 57.5 s


In [45]:
print(sk_negative_features.shape)
print(sk_positive_features.shape)
print(sk_test_neg_features.shape)
print(sk_test_pos_features.shape)

(1706710, 8)
(1013358, 8)
(426679, 8)
(53336, 8)


1. concatenate features of positve and negative train
2. concatenate features pos and neg test
3. add labels 1 or 0 
4. suffle !
5. Use decision tree/random forest or Xgboost

In [49]:
label_train_pos = np.ones(sk_positive_features.shape[0])
label_train_neg = np.zeros(sk_negative_features.shape[0])
label_test_pos = np.ones(sk_test_pos_features.shape[0])
label_test_neg = np.zeros(sk_test_neg_features.shape[0])

In [58]:
indices = np.arange(X_train.shape[0])

In [59]:
rnd = np.random.RandomState(43)

In [60]:
rnd.shuffle(indices)

In [55]:
label_train = np.concatenate([label_train_pos, label_train_neg])
label_test = np.concatenate([label_test_pos, label_test_neg])
X_train = np.concatenate([sk_positive_features, sk_negative_features])
X_test = np.concatenate([sk_test_pos_features, sk_test_neg_features])

X_train = X_train[indices]
label_train = label_train[indices]

In [61]:
from sklearn.tree import DecisionTreeClassifier

In [90]:
tree = DecisionTreeClassifier(
    random_state = 42,
    max_depth=10,
).fit(X_train, label_train)


In [91]:
test_pred = tree.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

In [92]:
# training
train_pred = tree.predict(X_train)
print("Training: ")
print("Accuracy: ", accuracy_score(label_train, train_pred))
print("matthews_corrcoef: ", matthews_corrcoef(label_train, train_pred))

Training: 
Accurarcy:  0.9096191712854238
matthews_corrcoef:  0.8051796655510743


In [93]:
# test 
print("Test:")
accuracy_score(label_test, test_pred)

Test:


0.9509182004729019

In [94]:
matthews_corrcoef(label_test, test_pred)

0.7581923199189939